In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import numpy as np

# =====================================
# 1) DEVICE
# =====================================
if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

# =====================================
# 2) CHARGER TOKENIZER
# =====================================
tokenizer = AutoTokenizer.from_pretrained(
    "qwen2_lora_multimodal",
    trust_remote_code=True
)
tokenizer.padding_side = "left"
tokenizer.pad_token = tokenizer.eos_token

# =====================================
# 3) CHARGER LE MODÈLE DE BASE
# =====================================
base_model = AutoModelForCausalLM.from_pretrained(
    "Qwen/Qwen2-1.5B",
    dtype=torch.float16,          # pour transformers >=4.57
    trust_remote_code=True
).to(device)

# =====================================
# 4) CHARGER LoRA FINETUNÉ
# =====================================
model = PeftModel.from_pretrained(
    base_model,
    "qwen2_lora_multimodal"
).to(device)

model.eval()

# =====================================
# 5) VisionAdapter (doit matcher EXACTEMENT celui du training)
# =====================================
class VisionAdapter(torch.nn.Module):
    def __init__(self, input_dim=1024, hidden=4096,
                 llm_dim=base_model.config.hidden_size,
                 num_tokens=64):
        super().__init__()
        self.num_tokens = num_tokens
        self.llm_dim = llm_dim
        self.mlp = torch.nn.Sequential(
            torch.nn.Linear(input_dim, hidden),
            torch.nn.ReLU(),
            torch.nn.Linear(hidden, num_tokens * llm_dim)
        )

    def forward(self, x):
        out = self.mlp(x)  # (B, num_tokens * hidden)
        return out.view(x.size(0), self.num_tokens, self.llm_dim)

# Charger l’adapter
adapter = VisionAdapter().to(device)
adapter.load_state_dict(torch.load("vision_adapter.pt", map_location=device))
adapter.eval()

# =====================================
# 6) Fonction pour injecter les tokens visuels (INFÉRENCE)
# =====================================
def inject_visual_tokens_for_inference(adapter, embed, model, tokenizer):
    # embed: (1, 1024)
    with torch.no_grad():
        vis_tokens = adapter(embed)                    # (1, 64, hidden)

    # Choisir un token de départ : bos si dispo, sinon eos, sinon pad
    bos_id = tokenizer.bos_token_id
    if bos_id is None:
        if tokenizer.eos_token_id is not None:
            bos_id = tokenizer.eos_token_id
        else:
            bos_id = tokenizer.pad_token_id

    dummy = torch.tensor([[bos_id]], device=device)

    text_embedding_layer = model.get_input_embeddings()
    dummy_text_emb = text_embedding_layer(dummy)       # (1, 1, hidden)

    # aligner dtype (LLM en float16)
    vis_tokens = vis_tokens.to(dtype=dummy_text_emb.dtype)

    # concaténation visuels + dummy
    full = torch.cat([vis_tokens, dummy_text_emb], dim=1)  # (1, 64+1, hidden)

    # mask : 1 sur tout
    mask_vis = torch.ones((1, vis_tokens.size(1)), device=device, dtype=torch.long)
    mask_dummy = torch.ones((1, 1), device=device, dtype=torch.long)
    full_mask = torch.cat([mask_vis, mask_dummy], dim=1)   # (1, 64+1)

    return full, full_mask

# =====================================
# 7) Charger un embedding (ex: index 0)
# =====================================
embeddings = np.load("sentinel_embeddings_1024.npy")   # (N, 1024)
embed = torch.tensor(embeddings[0], dtype=torch.float32).unsqueeze(0).to(device)

# =====================================
# 8) Préparer le prompt texte
# =====================================
prompt = (
            "Analyse l’état agricole de la parcelle à partir de l’embedding visuel.\n"
            "Décris en quelques phrases :\n"
            "- la vigueur de la végétation\n"
            "- l’humidité du couvert\n"
            "- la proportion sol/végétation\n"
            "- la biomasse\n"
            "- l’état général de la croissance\n"
        )
input_ids = tokenizer(prompt, return_tensors="pt").input_ids.to(device)

text_embedding_layer = model.get_input_embeddings()
text_embeds = text_embedding_layer(input_ids)         # (1, L, hidden)

# =====================================
# 9) Injecter les tokens visuels
# =====================================
vis_embeds, vis_mask = inject_visual_tokens_for_inference(adapter, embed, model, tokenizer)

# concat visuels + texte
inputs_embeds = torch.cat([vis_embeds, text_embeds], dim=1)  # (1, 64+L, hidden)

# construire le mask complet
mask_text = torch.ones_like(input_ids, dtype=torch.long)
attention_mask = torch.cat([vis_mask, mask_text], dim=1)     # (1, 64+L)

# =====================================
# 10) GÉNÉRATION
# =====================================
with torch.no_grad():
    output = model.generate(
        inputs_embeds=inputs_embeds,
        attention_mask=attention_mask,
        max_new_tokens=1024,
        do_sample=False
    )

# =====================================
# 11) AFFICHAGE RÉSULTAT
# =====================================
text = tokenizer.decode(output[0], skip_special_tokens=True)
print("\n===== RÉPONSE DU MODÈLE =====\n")
print(text)

/opt/anaconda3/lib/python3.12/site-packages/bitsandbytes/cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.



===== RÉPONSE DU MODÈLE =====

L’analyse spectrale montre une végétation modérée et cultures en croissance active. L’indice d’humidité suggère couvert végétal bien hydraté. Selon le SAVI, mélange sol + végétation. L’EVI indique biomasse élevée. L’ARVI montre végétation relativement stable malgré l'atmosphère. Le DVI confirme activité végétale moyenne. Le ATVI indique végétation relativement stable malgré l'atmosphère.
